In [ ]:
# import packages
import numpy as np
import random

import pandas as pd
import plotly.express as px

In [ ]:
# set constants
L = 0.05 # length
H = 0.05 # height
dx = 0.0025
dy = 0.0025
tmax = 10
dt = 0.01 # change in time
epsilon = 0.0001

In [5]:
# what is the Fo number?
NN = 1000

# generate the random noise values
domainalp = np.array([])
for i in range(NN):
    domainalp = np.append(domainalp, 0.5e-5 + 1e-5 * random.random())

print(domainalp.shape)

CENTERT = [] # NN x 1
AVET = [] # NN x 1

# list of simulated data matrices
SimulateData = [] # NN x 10
SimulateDataNoNoise = []

(1,)


In [6]:
for iN in range(NN):
    # generate the noise values
    r_x = domainalp[iN] * dt/(pow(dx, 2))
    r_y = domainalp[iN] * dt/(pow(dy, 2))
    fo = r_x + r_y

    # what does this condition mean? no convergence?
    if fo > 0.5000:
        raise Exception("Numerical stability requires Fo <= 1/2")
        exit()

    # we create a matrix of values
    nx = (int) (L / dx + 1)
    ny = (int) (H / dx + 1)

    grid = np.empty((nx, ny))

    # find center
    ic = (int) ((nx - 1) / 2) # no plus one, index starts at 0
    jc = (int) ((ny - 1) / 2)

    # set boundary conditions
    T = np.zeros((nx, ny))

    # column 0
    # T[:,0] = 1

    # column ny - 1
    T[:, ny-1] = 1

    # row 0
    T[0, :] = 1

    # row nx - 1
    T[nx - 1, :] = 1

    # I had difficulty telling what this step does?? are they not 0 and 1?)
    Tmin = np.min(np.min(T)) 
    Tmax = np.max(np.max(T)) 
    
    # iterative component
    n = 0 # index

    # nmax = (int) (tmax/dt)
    nmax = 1000

    CenterT = [] # center temperature
    AveT = []
    SaveIndex = 0
    iSave = 0
    bSaveImage = 0

    while n < nmax:
            n = n + 1
            SaveIndex = SaveIndex + 1

            T_n = T

            # define temperature values
            for j in range(1, nx - 1):
                for i in range(1, nx - 1):
                     T[j][i]= T_n[j][i] + r_x*(T_n[j][i+1]-2*T_n[j][i]+T_n[j][i-1]) + r_y*(T_n[j+1][i]-2*T_n[j][i]+T_n[j-1][i])

            a = T.shape
            NoiseT = T + 0.1 * np.random.rand(a[0], a[1])

    fig = px.imshow(NoiseT, text_auto=True, aspect="auto")
    fig.show()